## <strong> Importation des modules </strong>

In [1]:
import warnings
warnings.filterwarnings('ignore')

########################################

import pandas as pd
import numpy as np
from scipy import stats

#########################################

import seaborn as sns
#%matplotlib inline
#%matplotlib notebook
sns.set()

#########################################

import import_ipynb
from P5_01_Mes_fonctions import *

importing Jupyter notebook from P5_01_Mes_fonctions.ipynb


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## <strong> A - Importation des données </strong>

## <strong> A.1 - </strong> liste de tous les <strong> produits vendus par OLIST </strong>

In [2]:
data_product_list = pd.read_csv("products_dataset.csv")

In [3]:
data_product_list.head(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


## <strong> A.2 - </strong>  Liste des <strong> catégories de produits vendus :</strong>  équivalent portugais en anglais 

In [4]:
data_products_category = pd.read_csv("product_category_name_translation.csv")

In [5]:
data_products_category.head(2)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


### <strong> A.2.1 - Conversion de la catégorie des produits en langue anglaise </strong>

In [6]:
data_product_list.replace( data_products_category["product_category_name"].values , data_products_category["product_category_name_english"].values , inplace = True )

In [7]:
data_product_list.head(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0


In [8]:
data_product_list["product_volume_litre"] = data_product_list[["product_length_cm","product_height_cm","product_width_cm"]].prod(axis=1)/1000

In [9]:
data_product_list.drop(["product_name_lenght","product_description_lenght","product_length_cm","product_height_cm","product_width_cm","product_weight_g"],axis=1,inplace=True)

In [10]:
data_product_list.head(2)

,product_id,product_category_name,product_photos_qty,product_volume_litre
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,1.0,2.24
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1.0,10.80


In [11]:
del data_products_category

## <strong> A.3 - </strong> Liste de toutes les <strong> données de géolocalisation </strong>

In [12]:
data_geolocation = pd.read_csv("geolocation_dataset.csv")

In [13]:
data_geolocation.head(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


Suppression de duplicatats

In [14]:
data_geolocation.drop_duplicates( ["geolocation_zip_code_prefix","geolocation_city","geolocation_state"] ,keep='first' , inplace =True )

## <strong> A.4 - </strong> Liste de tous les <strong> vendeurs  ayant exécuté toutes les commandes passées chez Olist </strong>

In [15]:
data_sellers = pd.read_csv("sellers_dataset.csv")

In [16]:
data_sellers.head(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP


In [17]:
data_sellers.shape

(3095, 4)

### <strong> A.4.1 -  Ajout des coordonnées geographiques des vendeurs </strong>

In [18]:
data_sellers = pd.merge( data_sellers , data_geolocation , left_on= ["seller_zip_code_prefix","seller_city","seller_state"], 
                        right_on=["geolocation_zip_code_prefix","geolocation_city","geolocation_state"] , how ="left" )

### Suppression de colonnes inutiles

In [19]:
data_sellers.drop(["geolocation_zip_code_prefix","geolocation_city","geolocation_state"],axis=1, inplace=True)

### Rename des variables de geolocalisation

In [20]:
data_sellers.rename(columns={ "geolocation_lat":"sellers_geoloc_lat" , "geolocation_lng":"sellers_geoloc_long" }, inplace=True)

In [21]:
data_sellers.head(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,sellers_geoloc_lat,sellers_geoloc_long
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.898536,-47.063125
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.380500,-46.952965


In [22]:
data_sellers.shape

(3095, 6)

## <strong> A.5 - </strong> Liste de toutes les <strong> options de payements de tous les achats </strong>

In [23]:
data_order_payments = pd.read_csv("order_payments_dataset.csv")

In [24]:
data_order_payments.rename(columns={"payment_sequential":"Nbr_mode_de_paiement","payment_installments":"Nbr_de_versement"},inplace=True)

In [25]:
data_order_payments = data_order_payments.groupby(["order_id"], as_index=False).agg({"Nbr_mode_de_paiement":"max", "payment_type": series_modes, "payment_value":"sum"} )

In [26]:
data_order_payments.head(2)

,order_id,Nbr_mode_de_paiement,payment_type,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,credit_card,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,credit_card,259.83


## <strong> A.6 - </strong> Liste de tous les <strong> acheteurs/clients et leurs emplacement </strong>

In [27]:
data_customers = pd.read_csv("customers_dataset.csv")

In [28]:
data_customers.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [29]:
data_customers.shape

(99441, 5)

### <strong> A.6.1 -  Ajout des coordonnées geographiques des clients </strong>

In [30]:
data_customers = pd.merge( data_customers , data_geolocation , left_on= ["customer_zip_code_prefix","customer_city","customer_state"], 
                        right_on=["geolocation_zip_code_prefix","geolocation_city","geolocation_state"] , how ="left" )

### Suppression de colonnes inutiles

In [31]:
data_customers.drop( ["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"], axis=1, inplace=True )

### Rename des variables de geolocalisation

In [32]:
data_customers.rename(columns={ "geolocation_lat":"customers_geoloc_lat" , "geolocation_lng":"customers_geoloc_long" }, inplace=True)

In [33]:
data_customers.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customers_geoloc_lat,customers_geoloc_long
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.509897,-47.397866
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.726853,-46.545746


In [34]:
data_customers.shape

(99441, 7)

## <strong> A.7 - </strong> Liste de tous les <strong> avis/retours de chaque clients </strong>

In [35]:
data_order_reviews = pd.read_csv("order_reviews_dataset.csv")

In [36]:
data_order_reviews.drop(["review_comment_title","review_comment_message","review_creation_date","review_answer_timestamp"],axis=1, inplace=True)

In [37]:
data_order_reviews.head(2)

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5


## <strong> A.8 - </strong> Liste de tous les <strong> articles achetés dans chaque commande </strong>

In [38]:
data_order_items = pd.read_csv("order_items_dataset.csv")

### <strong> A.8.1 - </strong> Rename des variables donnant la quantité d'articles

In [39]:
data_order_items.rename(columns={ "order_item_id":"order_items_quantity"}, inplace=True)

In [40]:
pd.Series([1,1,2,2]).mode().values

array([1, 2], dtype=int64)

### <strong> A.8.3 - </strong> Supression de la variable de temps

In [41]:
data_order_items.drop(["shipping_limit_date"], axis=1, inplace=True)

In [42]:
data_order_items = data_order_items.groupby(["order_id"], as_index=False).agg({"order_items_quantity":"sum","product_id":lambda x : x.mode()[0],
                                                                               "seller_id":lambda x : x.mode()[0] , "price":"mean"})

In [43]:
data_order_items.rename(columns={"price":"items_mean_price"} ,inplace=True)

In [44]:
data_order_items.head(2)

,order_id,order_items_quantity,product_id,seller_id,items_mean_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.9


## <strong> A.9 - </strong> Liste de toutes les <strong> données de base de chaque commande </strong>

In [45]:
data_orders = pd.read_csv("orders_dataset.csv")

### <strong> A.9.1 - </strong> Rename des variables de temps 

In [46]:
data_orders.rename( columns={"order_purchase_timestamp":"date_achat_client", "order_delivered_customer_date":"date_reception_cmd",
                             "order_estimated_delivery_date":"date_reception_théorique"} ,inplace=True )

### <strong> A.9.2 - </strong> Formatage des dates

In [47]:
for i in ["date_achat_client","date_reception_cmd","date_reception_théorique"] :
    data_orders[i] = pd.to_datetime(data_orders[i] , yearfirst=True)

### <strong> A.9.3 - </strong> supression des variables

In [48]:
data_orders.drop(["order_delivered_carrier_date","order_status","order_approved_at"], axis=1, inplace = True)

In [49]:
data_orders.head(2)

,order_id,customer_id,date_achat_client,date_reception_cmd,date_reception_théorique
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13


## <strong> B - Construction du Dataframe global </strong>

In [50]:
data = pd.merge(data_orders , data_order_payments , on="order_id" , how = "left")
data = pd.merge(data , data_order_reviews , on="order_id" , how = "left").drop(["review_id"] , axis=1)
data = pd.merge(data , data_customers , on="customer_id" , how = "left").drop(["customer_id"] , axis=1)
data = pd.merge(data , data_order_items , on="order_id" , how = "left")
data = pd.merge(data , data_product_list , on="product_id" , how = "left").drop(["product_id"], axis=1)
data = pd.merge(data , data_sellers , on="seller_id" , how = "left").drop(["seller_id"], axis=1)

### <strong> B.1 - </strong> supression des achats à valeurs nulles

In [51]:
if "payment_value" in data.columns :
    data = data[data["payment_value"]!= 0]

In [52]:
data

,order_id,date_achat_client,date_reception_cmd,date_reception_théorique,Nbr_mode_de_paiement,payment_type,payment_value,review_score,customer_unique_id,customer_zip_code_prefix,...,order_items_quantity,items_mean_price,product_category_name,product_photos_qty,product_volume_litre,seller_zip_code_prefix,seller_city,seller_state,sellers_geoloc_lat,sellers_geoloc_long
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,3.0,voucher,38.71,4.0,7c396fd4830fd04220f754e42b4e5bff,3149,...,1.0,29.99,housewares,4.0,1.976,9350.0,maua,SP,-23.680114,-46.452454
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,1.0,boleto,141.46,4.0,af07308b275d755c9edb36a90c618231,47813,...,1.0,118.70,perfumery,1.0,4.693,31570.0,belo horizonte,SP,NaN,NaN
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,1.0,credit_card,179.12,5.0,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,159.90,auto,1.0,9.576,14840.0,guariba,SP,-21.362358,-48.232976
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,1.0,credit_card,72.20,5.0,7c142cf63193a1473d2e66489a9ae977,59296,...,1.0,45.00,pet_shop,3.0,6.000,31842.0,belo horizonte,MG,-19.840168,-43.923299
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,1.0,credit_card,28.62,5.0,72632f0f9dd73dfee390c9b22eb56dd6,9195,...,1.0,19.90,stationery,4.0,11.475,8752.0,mogi das cruzes,SP,-23.551707,-46.260979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99987,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,2017-03-17 15:08:01,2017-03-28,1.0,credit_card,85.08,5.0,6359f309b166b0196dbf7ad2ac62bb5a,12209,...,1.0,72.00,health_beauty,1.0,5.148,12913.0,braganca paulista,SP,-22.957505,-46.524886
99988,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,2018-02-28 17:37:56,2018-03-02,1.0,credit_card,195.00,4.0,da62f9e57a76d978d02ab5362c509660,11722,...,1.0,174.90,baby,4.0,16.000,17602.0,tupa,SP,-21.935321,-50.497562
99989,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,2017-09-21 11:24:17,2017-09-27,1.0,credit_card,271.01,5.0,737520a9aad80b3fbbdad19b66b37b30,45920,...,1.0,205.99,home_appliances_2,2.0,63.360,8290.0,sao paulo,SP,-23.551013,-46.448489
99990,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,2018-01-25 23:32:54,2018-02-15,1.0,credit_card,441.16,2.0,5097a5312c8b157bb7be58ae360ef43c,28685,...,3.0,179.99,computers_accessories,1.0,8.000,37175.0,ilicinea,MG,-20.944706,-45.827098


### <strong> B.2 - Evaluation de la  distance à vol d'oiseau entre le client et le vendeur </strong>

In [53]:
data["distance_client_vendeur_Km"] =  np.array( [distance_a_vol_oiseau( data.loc[i,"customers_geoloc_lat"] , data.loc[i,"sellers_geoloc_lat"]  ,
                                                               data.loc[i,"customers_geoloc_long"] , data.loc[i,"sellers_geoloc_long"] ) for i in data.index ] )

### <strong> B.3 - </strong> Supression des coordonnées geographiques

In [54]:
data.drop(["customers_geoloc_lat","sellers_geoloc_lat","customers_geoloc_long","sellers_geoloc_long","customer_zip_code_prefix","customer_state",
           "seller_zip_code_prefix","seller_state"] , axis = 1, inplace=True)

## <strong> C - Construction des durées </strong>

In [55]:
data["durée_reception_cmd_jr"] = data["date_reception_cmd"] - data["date_achat_client"]
data["durée_reception_respectée_jr"] = data["date_reception_cmd"] -data["date_reception_théorique"]
data["date_achat_client"] = data["date_achat_client"] -  pd.to_datetime("01 october 2016" ,yearfirst=True)

In [56]:
for j in [ k for k in data.columns if ("durée_" in k ) or ("date_achat_client" == k )] :
    data[j] = np.array([  data.loc[i,j].total_seconds() for i in data.index ])/(3600.*24.)

In [57]:
duration = [k for k in data.columns if "durée_" in k]
data[duration] = data[duration].astype(np.float64)

In [58]:
data.sort_values(by= "date_achat_client" ,ignore_index=True, inplace=True)

### <strong> C.1 - </strong> Supression des variables de temps

In [59]:
data.drop([i for i in data.columns if ("date_" in i)&("_achat_client" not in i)] , axis = 1, inplace=True)

## <strong> D -  Données de fréquence de maintenance </strong>

In [60]:
data_freq = data.groupby(["customer_unique_id"], as_index=False).agg({"date_achat_client" : "min", "order_id":"count", "Nbr_mode_de_paiement" :"mean", "payment_value" : "sum", 
                                                                      "review_score" : "mean", "order_items_quantity" : "sum", "items_mean_price":"mean" ,
                                                                      "distance_client_vendeur_Km":"mean", "durée_reception_cmd_jr":"mean","durée_reception_respectée_jr":"mean",
                                                                      "payment_type": series_modes, "customer_city": series_modes, "seller_city": series_modes
                                                                      } ).drop("customer_unique_id",axis=1)

## <strong> E - Données de modélisation</strong>

In [61]:
data_mod = data.groupby(["customer_unique_id"], as_index=False).agg({"date_achat_client" : f, "order_id":"count", "Nbr_mode_de_paiement" :"mean", "payment_value" : "sum", 
                                                                      "review_score" : "mean", "order_items_quantity" : "sum", "items_mean_price":"mean" ,
                                                                      "distance_client_vendeur_Km":"mean", "durée_reception_cmd_jr":"mean","durée_reception_respectée_jr":"mean",
                                                                      "payment_type": series_modes, "customer_city": series_modes, "seller_city": series_modes
                                                                      } ).drop("customer_unique_id",axis=1)

In [62]:
data_mod.rename(columns={"date_achat_client":"Jour_moyen_entre_achats" , "order_id":"Nombre_commandes"}, inplace=True)
data_freq.rename( columns ={ "order_id":"Nombre_commandes" } , inplace = True )

In [63]:
data_mod.dropna(axis='index',inplace=True)
data_freq.dropna(axis='index',inplace=True)

In [64]:
data.head()

,order_id,date_achat_client,Nbr_mode_de_paiement,payment_type,payment_value,review_score,customer_unique_id,customer_city,order_items_quantity,items_mean_price,product_category_name,product_photos_qty,product_volume_litre,seller_city,distance_client_vendeur_Km,durée_reception_cmd_jr,durée_reception_respectée_jr
0,2e7a8482f6fb09756ca50c10d7bfc047,-26.114363,1.0,credit_card,136.23,1.0,b7d76e111c89f7ebf14761390f0f7d17,boa vista,3.0,36.445,furniture_decor,2.0,5.376,monte siao,2828.774821,NaN,NaN
1,e5fa5a7210941f7d56d0208e4e071d35,-25.989190,1.0,credit_card,75.06,1.0,4854e9b3feff728c13ee5fc7d1547e92,passo fundo,1.0,59.500,telephony,1.0,1.250,curitiba,345.147657,NaN,NaN
2,809a282bbd5dbcabb6f2f724fca862ec,-17.358113,1.0,credit_card,40.95,1.0,009b0127b727ab0ba422f6d9604487c7,sao jose dos campos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bfbd0f9bdef84302105ad712db648a6c,-15.488449,NaN,NaN,NaN,1.0,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,6.0,44.990,health_beauty,1.0,4.096,curitiba,551.529866,54.813194,36.324745
4,71303d7e93b399f5bcd537d124c0bcfa,1.922130,1.0,credit_card,109.34,1.0,0eb1ee9dba87f5b36b4613a65074337c,sao paulo,1.0,100.000,baby,1.0,5.832,sao paulo,10.519797,NaN,NaN


In [65]:
data_mod.head()

,Jour_moyen_entre_achats,Nombre_commandes,Nbr_mode_de_paiement,payment_value,review_score,order_items_quantity,items_mean_price,distance_client_vendeur_Km,durée_reception_cmd_jr,durée_reception_respectée_jr,payment_type,customer_city,seller_city
0,0.0,1,1.0,141.90,5.0,1.0,129.90,77.693891,6.411227,-4.132905,credit_card,cajamar,piracicaba
1,0.0,1,1.0,27.19,4.0,1.0,18.90,12.543108,3.285590,-4.248125,credit_card,osasco,sao paulo
2,0.0,1,1.0,86.22,3.0,1.0,69.00,490.603272,25.731759,-1.389734,credit_card,sao jose,campo limpo paulista
3,0.0,1,1.0,43.62,4.0,1.0,25.99,2474.833460,20.037083,-11.108970,credit_card,belem,sao paulo
4,0.0,1,1.0,196.89,5.0,1.0,180.00,69.779397,13.141134,-7.035463,credit_card,sorocaba,jacarei


In [66]:
data_freq.head()

,date_achat_client,Nombre_commandes,Nbr_mode_de_paiement,payment_value,review_score,order_items_quantity,items_mean_price,distance_client_vendeur_Km,durée_reception_cmd_jr,durée_reception_respectée_jr,payment_type,customer_city,seller_city
0,586.455868,1,1.0,141.90,5.0,1.0,129.90,77.693891,6.411227,-4.132905,credit_card,cajamar,piracicaba
1,583.466285,1,1.0,27.19,4.0,1.0,18.90,12.543108,3.285590,-4.248125,credit_card,osasco,sao paulo
2,160.878507,1,1.0,86.22,3.0,1.0,69.00,490.603272,25.731759,-1.389734,credit_card,sao jose,campo limpo paulista
3,376.853947,1,1.0,43.62,4.0,1.0,25.99,2474.833460,20.037083,-11.108970,credit_card,belem,sao paulo
4,409.823403,1,1.0,196.89,5.0,1.0,180.00,69.779397,13.141134,-7.035463,credit_card,sorocaba,jacarei


In [67]:
data_mod.shape

(90018, 13)

In [68]:
data.shape

(99989, 17)

In [69]:
data_freq.shape

(90018, 13)

## <strong> F - Sauvegarde des données </strong>

In [70]:
data.to_csv("data_merge.csv")
data_mod.to_csv("data_mod.csv")
data_freq.to_csv("data_freq.csv")
print("Sauvegarde des documents terminée avec succès . . .")

Sauvegarde des documents terminée avec succès . . .
